# Explanatory Analysis

Set up libraries

In [2]:
import scipy
from datetime import date
import polars as pl
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
import plotly.io as pio
pio.templates.default = 'plotly_white'


In [4]:
data_link = "./Data/data.csv"
df = pl.read_csv(data_link).with_columns(pl.col('week_start').cast(pl.Date)).sort('week_start')

df.head(4)

week_start,new_customers,promo_investment,ppc_brand_investment,ppc_generic_investment,facebook_investment,instagram_investment,year,week,promo_investment_lag0,ppc_brand_investment_lag0,ppc_generic_investment_lag3,facebook_investment_lag2,instagram_investment_lag6,time,sin_52_1,cos_52_1,sin_52_2,cos_52_2
date,i64,f64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64
2020-05-18,777,1304.227029,756.818857,1512.071585,297.193399,4593.059512,2020,21,1304.227029,756.818857,2300.295763,999.081772,82.890357,113,0.885456,0.464723,0.822984,-0.568065
2020-05-25,978,3000.504127,69.654279,399.82446,0.0,792.564928,2020,22,3000.504127,69.654279,4268.329032,637.97411,1246.32403,114,0.935016,0.354605,0.663123,-0.748511
2020-06-01,950,4609.816216,357.467625,471.992936,1769.420849,3281.127107,2020,23,4609.816216,357.467625,4219.098648,297.193399,1969.201111,115,0.970942,0.239316,0.464723,-0.885456
2020-06-08,800,3429.065037,425.663469,1181.626485,2353.024023,4510.560793,2020,24,3429.065037,425.663469,1512.071585,0.0,2889.605271,116,0.992709,0.120537,0.239316,-0.970942


### Quality Check 	✅

In [11]:
def check_complete_datarange(df: pl.DataFrame,
                             date_col: str = 'week_start'):

  min_date = df.select(pl.col(date_col).min()).item()
  max_date = df.select(pl.col(date_col).max()).item()
  date_range = pl.date_range(start=min_date, end=max_date, interval="1w",
                             eager=True).to_list()
  print(set(date_range))
  assert set(date_range) == set(df[date_col].to_list())


In [ ]:
check_complete_datarange(df)

In [ ]:
df.describe()

### Know your target ⭕

In [6]:
df.columns

['week_start',
 'new_customers',
 'promo_investment',
 'ppc_brand_investment',
 'ppc_generic_investment',
 'facebook_investment',
 'instagram_investment',
 'year',
 'week',
 'promo_investment_lag0',
 'ppc_brand_investment_lag0',
 'ppc_generic_investment_lag3',
 'facebook_investment_lag2',
 'instagram_investment_lag6',
 'time',
 'sin_52_1',
 'cos_52_1',
 'sin_52_2',
 'cos_52_2']

In [7]:
target = "new_customers"
date_col = 'week_start'
channels = ['promo_investment',
            'ppc_brand_investment',
            'ppc_generic_investment',
            'facebook_investment',
            'instagram_investment']

### Trend Analysis 📈

In [10]:
rolling_window_widget = widgets.IntSlider(value=1,
    min=1,
    max=40,
    step=1,
    description='Window')


def update_rolling_plot(window):

  base_title = f"{target} Evolution Over Time"
  title = base_title if window == 1 else base_title +  f"- Rolling Mean (Window = {window})"

  rolled_df = df.with_columns(pl.col(target).rolling_mean(window))
  fig = px.line(rolled_df, x=date_col, y=target,
                title=title)
  #fig.update_layout(width=1300, height=400)
  fig.show(renderer="colab")

interactive_plot = interactive(update_rolling_plot,
                               window=rolling_window_widget)

display(interactive_plot)

interactive(children=(IntSlider(value=1, description='Window', max=40, min=1), Output()), _dom_classes=('widge…

🔍 Observations <br>
⬛ Increasing trend over time, except in 2023, where sales show a decline. <br>
⬛ Presence of seasonality.

### Seasonality Analysis 📈

In [15]:
fig = px.line(df.with_columns(pl.col(target).rolling_mean(5)),
        x='week',
        y=target,
        color = 'year',
        hover_data=[date_col],
        title='New Customers Comparison Over Years')

fig.show()

🔍 Observations 

⬛ In 2020, the summer spike in new customers begins at the end of July. <br>
⬛ In 2021, the summer spike shifts earlier, starting at the end of May. <br>
⬛ In 2022, the summer spike occurs even earlier, at the beginning of May. <br>
⬛ In 2023, an unusual decrease in new customers is observed at the beginning of June. <br>
⬛ This pattern suggests that a potential yearly seasonality is disrupted by the lingering effects of COVID and an anomalous event in 2023, making it less clean.

In [19]:
def compute_autocorrelation(df: pl.DataFrame,
                              col_name: str,
                              max_lag: int = 60,
                              sort_col: str = None) -> pl.DataFrame:
    # Optional sorting
    if sort_col:
        df = df.sort(sort_col)

    # Compute autocorrelations for each lag
    corrs = []
    for lag in range(0, max_lag + 1):
        shifted_df = df.with_columns(pl.col(col_name).shift(lag).alias(f"{col_name}_lag"))
        corr_value = shifted_df.drop_nulls().select(pl.corr(col_name, f"{col_name}_lag")).item()
        corrs.append(corr_value)

    # Return the results as a Polars DataFrame
    return pl.DataFrame({
        'Lag': range(0, max_lag + 1),
         col_name: corrs
    })


result = compute_autocorrelation(df, col_name=target, max_lag=60, sort_col=date_col)
fig=px.line(result, x='Lag', y=target, title=f"{target} - Autocorrelation Function")
fig.update_layout(yaxis_title="Correlation")
fig.show()

🔍 Observations <br>

The prominent correlation peak at lag 52 further confirms the presence of yearly seasonality. 

### New customers 💁‍♀️💁‍♂️ vs. Investments 💰

Timing matters ⏰

In [25]:
def scale_columns_expr(col_names: list[str]) -> pl.Expr:
  return [((pl.col(col) - pl.col(col).min()) / (pl.col(col).max() - pl.col(col).min())).alias(f"{col}_scaled")
          for col in col_names]


def shift_col_expr(col_names: list[str], lag: int) -> pl.Expr:
  return [pl.col(col).shift(lag).alias(col) for col in col_names]


def add_season_column(df: pl.DataFrame, date_col: str,
                      new_col: str = "season") -> pl.DataFrame:

    return df.with_columns(
        pl.when(pl.col(date_col).dt.month().is_in([12, 1, 2]))
        .then(pl.lit("Winter"))
        .when(pl.col(date_col).dt.month().is_in([3, 4, 5]))
        .then(pl.lit("Spring"))
        .when(pl.col(date_col).dt.month().is_in([6, 7, 8]))
        .then(pl.lit("Summer"))
        .otherwise(pl.lit("Autumn"))
        .alias(new_col)
    )

In [21]:
def plot_target_channel_relationship(df: pl.DataFrame,
                                     col_to_scale: str,
                                     col_to_corr: str,
                                     plot_title: str,
                                     target: str = target,
                                     ) -> None:
  df_scaled = df.with_columns(scale_columns_expr([target, col_to_scale]))
  corr_coeff = df_scaled.select(pl.corr(target, col_to_corr)).item()
  corr_coeff = np.round(corr_coeff, 2)


  fig = make_subplots(rows=1, cols=2, subplot_titles=("Normalized Line Plot",
                                                      f"Scatter Plot - Corr: {corr_coeff}"))

  fig.add_trace(
      go.Scatter(x=df_scaled[date_col].to_list(),
                 y=df_scaled[f"{target}_scaled"].to_list(),
                 mode='lines', name=target),
      row=1, col=1
  )
  fig.add_trace(
      go.Scatter(x=df_scaled[date_col].to_list(),
                 y=df_scaled[f"{col_to_scale}_scaled"].to_list(),
                 mode='lines', name=col_to_scale),
      row=1, col=1
  )
  fig.add_trace(
      go.Scatter(x=df_scaled[col_to_corr].to_list(),
                 y=df_scaled[target].to_list(),
                 mode='markers', showlegend=False),
      row=1, col=2
  )

  fig.update_layout(
      title=plot_title
  )
  fig.show()

In [28]:
lag_widget = widgets.IntSlider(value=0,
    min=0,
    max=8,
    step=1,
    description='Lag')

channels_dropdown = widgets.Dropdown(description='Channel', options=channels)
def plot_target_channel_relationship_on_lag_change(channel: str,
                                                   lag: int):

    df_shifted = (
        df
        .sort(date_col)
        .with_columns(shift_col_expr([channel], lag))
        .filter(pl.col(channel).is_not_null())

    )

    plot_title = f"{target} vs {channel} Relationship - Lag {lag}"
    plot_target_channel_relationship(df_shifted,
                                     col_to_scale=channel,
                                     col_to_corr=channel,
                                     plot_title=plot_title)

In [29]:
interactive_relationship_plot = interactive(plot_target_channel_relationship_on_lag_change,
                                            channel = channels_dropdown,
                                            lag=lag_widget
                                            )

display(interactive_relationship_plot)

interactive(children=(Dropdown(description='Channel', options=('promo_investment', 'ppc_brand_investment', 'pp…

🔍 Observations <br>

⬛ 🧐: the target saturates beyond a certain threshold of investment, indicating diminishing returns. <br>
⬛ For all other channels, except PPC brand, we observe a stronger correlation when the investments are lagged, suggesting a delayed effect on the target. <br>
⬛ Additionally, in most cases, the relationship between investments and the target is not perfectly linear, highlighting the need for transformations or advanced modeling techniques to capture these dynamics effectively.

### Quantitative Analysis - Cross Correlation 
⬛ Cross Correlation <br>
⬛ Spearman Correlation

In [30]:
def compute_cross_correlation(df: pl.DataFrame,
                              target: str,
                              feature: str,
                              max_lag: int,
                              corr_method: str
                              ) -> pl.DataFrame:

    corrs = []
    for lag in range(0, max_lag + 1):
        shifted_df = df.with_columns(pl.col(feature).shift(lag).alias(f"{feature}_lag"))
        corr_value = shifted_df.drop_nulls().select(pl.corr(target, f"{feature}_lag",
                                                            method=corr_method)).item()
        corrs.append(corr_value)

    return pl.DataFrame({
        'Lag': range(0, max_lag + 1),
         feature: corrs
    })


def get_cross_correlation_df(df: pl.DataFrame,
                             target: str = target,
                             channels: list[str] = channels,
                             max_lag: int = 12,
                             sort_col: str = date_col,
                             corr_method: str = 'pearson'):

  sorted_df = df.sort(sort_col)
  cross_correlations_list = []
  for channel in channels:
    sel_df = sorted_df.select(target, channel)
    cross_correlations_list.append(
        compute_cross_correlation(sel_df, target, channel,
                                  max_lag, corr_method)
    )

  return pl.concat(cross_correlations_list, how="align")

In [31]:
spearman_cross_corr_df = get_cross_correlation_df(df, corr_method='spearman')
pearson_cross_corr_df = get_cross_correlation_df(df, corr_method='pearson')

In [32]:
fig=px.line(pearson_cross_corr_df, x='Lag', y=channels, title="Pearson Cross Correlation")
fig.show()

In [34]:
fig=px.line(spearman_cross_corr_df, x='Lag', y=channels, title="Spearman Cross Correlation")
fig.show()
     

🔍 Observations <br>

⬛ PPC --> immediate effect <br>
⬛ Instagram --> latency <br>

In [37]:
def get_best_lag(cross_correlation_df: pl.DataFrame,
                 lag_col: str = 'Lag') -> dict[str, int]:

  channels = [col for col in cross_correlation_df.columns if
              col != lag_col]
  lag_dict = {}
  for channel in channels:
    best_lag = (
        cross_correlation_df
        .filter(pl.col(channel) == pl.max(channel))
        .select(lag_col).item()
    )
    lag_dict[channel] = best_lag

  return lag_dict

In [38]:
spearman_lag_dict = get_best_lag(spearman_cross_corr_df)
spearman_lag_dict

{'promo_investment': 0,
 'ppc_brand_investment': 0,
 'ppc_generic_investment': 3,
 'facebook_investment': 2,
 'instagram_investment': 6}

In [39]:
pearson_lag_dict = get_best_lag(pearson_cross_corr_df)
pearson_lag_dict

{'promo_investment': 0,
 'ppc_brand_investment': 1,
 'ppc_generic_investment': 3,
 'facebook_investment': 3,
 'instagram_investment': 9}

In [40]:
chosen_lag_dict = spearman_lag_dict

In [41]:
lagged_channels_names = [f"{channel}_lag{lag}" for channel, lag in chosen_lag_dict.items()]

In [42]:
def lag_channels_at_best_lag(df: pl.DataFrame, best_lag_dict: dict[str, int],
                             sort_col: str = date_col) -> pl.DataFrame:
  lagged_df = df.sort(sort_col)
  for channel, lag in best_lag_dict.items():
    lagged_df = lagged_df.with_columns(pl.col(channel).shift(lag).alias(f"{channel}_lag{lag}"))
  return lagged_df

In [43]:
lagged_df = lag_channels_at_best_lag(df, chosen_lag_dict)
lagged_df.head()

week_start,new_customers,promo_investment,ppc_brand_investment,ppc_generic_investment,facebook_investment,instagram_investment,year,week,promo_investment_lag0,ppc_brand_investment_lag0,ppc_generic_investment_lag3,facebook_investment_lag2,instagram_investment_lag6,time,sin_52_1,cos_52_1,sin_52_2,cos_52_2
date,i64,f64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64
2020-05-18,777,1304.227029,756.818857,1512.071585,297.193399,4593.059512,2020,21,1304.227029,756.818857,null,null,null,113,0.885456,0.464723,0.822984,-0.568065
2020-05-25,978,3000.504127,69.654279,399.82446,0.0,792.564928,2020,22,3000.504127,69.654279,null,null,null,114,0.935016,0.354605,0.663123,-0.748511
2020-06-01,950,4609.816216,357.467625,471.992936,1769.420849,3281.127107,2020,23,4609.816216,357.467625,null,297.193399,null,115,0.970942,0.239316,0.464723,-0.885456
2020-06-08,800,3429.065037,425.663469,1181.626485,2353.024023,4510.560793,2020,24,3429.065037,425.663469,1512.071585,0.0,null,116,0.992709,0.120537,0.239316,-0.970942
2020-06-15,320,6679.522877,530.894558,1962.369223,2351.488377,3760.098884,2020,25,6679.522877,530.894558,399.82446,1769.420849,null,117,1.0,5.5100e-16,1.1000e-15,-1.0


### Seasonal Effects 🌻 🌞 🍃 ❄